## Feature Engineering

Our current dataset contains only a few couple features: Elo ratings, referee, and home and away team. That doesn't mean we throw out the rest of the data here. While we can't use the score or home/away xG to predict the match where that data was recorded, we can use it to create features for future matches. For example, xG registered in previous games is a good predictor of future performance. By creating a feature of all xG to date (for that season), we can turn this data into usable features for our model. 

In [1]:
import soccerdata as sd
import pandas as pd
import time
import requests
import pandas as pd
import numpy as np

import warnings

from datetime import datetime, timedelta


warnings.filterwarnings('ignore')

[08/23/24 17:47:28] INFO     No custom team name replacements found. You can configure these in       ]8;id=891367;file:///Users/lkimball/opt/anaconda3/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=398579;file:///Users/lkimball/opt/anaconda3/lib/python3.9/site-packages/soccerdata/_config.py#90\90]8;;\
                             /Users/lkimball/soccerdata/config/teamname_replacements.json.                         

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=275630;file:///Users/lkimball/opt/anaconda3/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=856522;file:///Users/lkimball/opt/anaconda3/lib/python3.9/site-packages/soccerdata/_config.py#196\196]8;;\
                             /Users/lkimball/soccerdata/config/league_dict.json.                                   

In [2]:
df = pd.read_csv('EPL_data.csv')
df.head()

,season,game,week,day,date,time,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,home_starters,away_starters,home_team_strength,away_team_strength
0,1920,2019-08-09 Liverpool-Norwich City,1,Fri,2019-08-09,20:00,liverpool,1.8,4–1,0.9,norwich city,Michael Oliver,928467bd,2050.401855,1634.853149,"['Alisson', 'Fabinho', 'Virgil van Dijk', 'Geo...","['Tim Krul', 'Max Aarons', 'Ben Godfrey', 'Gra...",1.103,0.000
1,1920,2019-08-10 Burnley-Southampton,1,Sat,2019-08-10,15:00,burnley,0.9,3–0,1.2,southampton,Graham Scott,34b99058,1685.844604,1677.967529,"['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...","['Angus Gunn', 'Jannik Vestergaard', 'Jack Ste...",0.016,0.152
2,1920,2019-08-10 Crystal Palace-Everton,1,Sat,2019-08-10,15:00,crystal palace,0.9,0–0,1.1,everton,Jonathan Moss,a802f51e,1746.714111,1773.843750,"['Vicente Guaita', 'Joel Ward', 'Patrick van A...","['Jordan Pickford', 'Michael Keane', 'Richarli...",0.173,0.281
3,1920,2019-08-17 Arsenal-Burnley,2,Sat,2019-08-17,12:30,arsenal,0.8,2–1,1.5,burnley,Mike Dean,ff7eda21,1876.900879,1696.790283,"['Bernd Leno', 'Sokratis Papastathopoulos', 'D...","['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...",0.782,0.016
4,1920,2019-08-17 Aston Villa-Bournemouth,2,Sat,2019-08-17,15:00,aston villa,2.0,1–2,1.6,bournemouth,Martin Atkinson,7ad0ed82,1614.175537,1691.470581,"['Tom Heaton', 'Neil Taylor', 'Douglas Luiz', ...","['Aaron Ramsdale', 'Steve Cook', 'Nathan Aké',...",-0.029,0.204


In [3]:
df = df.drop(['day', 'time', 'game'], axis=1)

In [4]:
df.head()

,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,home_starters,away_starters,home_team_strength,away_team_strength
0,1920,1,2019-08-09,liverpool,1.8,4–1,0.9,norwich city,Michael Oliver,928467bd,2050.401855,1634.853149,"['Alisson', 'Fabinho', 'Virgil van Dijk', 'Geo...","['Tim Krul', 'Max Aarons', 'Ben Godfrey', 'Gra...",1.103,0.000
1,1920,1,2019-08-10,burnley,0.9,3–0,1.2,southampton,Graham Scott,34b99058,1685.844604,1677.967529,"['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...","['Angus Gunn', 'Jannik Vestergaard', 'Jack Ste...",0.016,0.152
2,1920,1,2019-08-10,crystal palace,0.9,0–0,1.1,everton,Jonathan Moss,a802f51e,1746.714111,1773.843750,"['Vicente Guaita', 'Joel Ward', 'Patrick van A...","['Jordan Pickford', 'Michael Keane', 'Richarli...",0.173,0.281
3,1920,2,2019-08-17,arsenal,0.8,2–1,1.5,burnley,Mike Dean,ff7eda21,1876.900879,1696.790283,"['Bernd Leno', 'Sokratis Papastathopoulos', 'D...","['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...",0.782,0.016
4,1920,2,2019-08-17,aston villa,2.0,1–2,1.6,bournemouth,Martin Atkinson,7ad0ed82,1614.175537,1691.470581,"['Tom Heaton', 'Neil Taylor', 'Douglas Luiz', ...","['Aaron Ramsdale', 'Steve Cook', 'Nathan Aké',...",-0.029,0.204


## Expected Goals Conceded and Registers to Date

Below we are setting the xG to date at 0.0 for the first game of the season, and then summing the cummulative xG (sorted by team name).

In [5]:
# Assuming df is your original dataframe
# Create a unified dataframe for both home and away xG values
games = df[['season', 'date', 'home_team', 'home_xg', 'away_team', 'away_xg']].copy()

# Create two dataframes: one for the home teams and one for the away teams
home_xg_df = games[['season', 'date', 'home_team', 'home_xg']].rename(columns={'home_team': 'team', 'home_xg': 'xg'})
away_xg_df = games[['season', 'date', 'away_team', 'away_xg']].rename(columns={'away_team': 'team', 'away_xg': 'xg'})

# Concatenate the home and away dataframes
all_xg_df = pd.concat([home_xg_df, away_xg_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_xg_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of xG for each team, grouped by season
all_xg_df['xg_cumulative'] = all_xg_df.groupby(['team', 'season'])['xg'].cumsum()

# Shift the cumsum by one, so we don't include the xG of the current game
all_xg_df['xg_cumulative'] = all_xg_df.groupby(['team', 'season'])['xg_cumulative'].shift(fill_value=0)

# Now we have the cumulative xG for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_xg_df[['date', 'team', 'xg_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_cumulative': 'home_xG_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_xg_df[['date', 'team', 'xg_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_cumulative': 'away_xG_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_xG_to_date' and 'away_xG_to_date' columns with the correct cumulative xG values

In [8]:
df

,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,home_starters,away_starters,home_team_strength,away_team_strength,home_xG_to_date,away_xG_to_date
0,1920,1,2019-08-09,liverpool,1.8,4–1,0.9,norwich city,Michael Oliver,928467bd,2050.401855,1634.853149,"['Alisson', 'Fabinho', 'Virgil van Dijk', 'Geo...","['Tim Krul', 'Max Aarons', 'Ben Godfrey', 'Gra...",1.103,0.000,0.0,0.0
1,1920,1,2019-08-10,bournemouth,1.3,1–1,1.3,sheffield utd,Kevin Friend,d402cacd,1695.075562,1623.745361,"['Aaron Ramsdale', 'Steve Cook', 'Nathan Aké',...","['Dean Henderson', 'George Baldock', 'Enda Ste...",0.212,0.000,0.0,0.0
2,1920,1,2019-08-10,burnley,0.9,3–0,1.2,southampton,Graham Scott,34b99058,1685.844604,1677.967529,"['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...","['Angus Gunn', 'Jannik Vestergaard', 'Jack Ste...",0.016,0.152,0.0,0.0
3,1920,1,2019-08-10,crystal palace,0.9,0–0,1.1,everton,Jonathan Moss,a802f51e,1746.714111,1773.843750,"['Vicente Guaita', 'Joel Ward', 'Patrick van A...","['Jordan Pickford', 'Michael Keane', 'Richarli...",0.173,0.281,0.0,0.0
4,1920,1,2019-08-10,tottenham,2.4,3–1,0.7,aston villa,Chris Kavanagh,404ee5d3,1894.996338,1616.385376,"['Hugo Lloris', 'Danny Rose', 'Toby Alderweire...","['Tom Heaton', 'Neil Taylor', 'John McGinn', '...",0.621,-0.017,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2324,38,2024-05-19,crystal palace,2.5,5–0,0.9,aston villa,Darren Bond,c975c7a6,1742.560303,1787.528564,"['Dean Henderson', 'Tyrick Mitchell', 'Marc Gu...","['Robin Olsen', 'Diego Carlos', 'Ezri Konsa', ...",0.387,0.387,46.2,62.6
1896,2324,38,2024-05-19,liverpool,4.5,2–0,0.5,wolves,Chris Kavanagh,d4823ed5,1897.319702,1681.174927,"['Alisson', 'Wataru Endo', 'Virgil van Dijk', ...","['José Sá', 'Rayan Aït-Nouri', 'Santiago Bueno...",1.122,0.225,83.3,46.5
1897,2324,38,2024-05-19,luton town,2.0,2–4,1.1,fulham,Matt Donohue,0fde9d70,1572.322388,1707.862427,"['Thomas Kaminski', 'Gabriel Osho', 'Chiedozie...","['Bernd Leno', 'Kenny Tete', 'Calvin Bassey', ...",-0.017,0.207,40.3,49.7
1898,2324,38,2024-05-19,manchester city,1.9,3–1,0.4,west ham,John Brooks,29335211,2048.724609,1728.056885,"['Stefan Ortega', 'Kyle Walker', 'Rúben Dias',...","['Alphonse Areola', 'Aaron Cresswell', 'Kurt Z...",1.685,0.455,78.7,52.1


In [9]:
#Checking if End of season xG values are correct


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,home_starters,away_starters,home_team_strength,away_team_strength,home_xG_to_date,away_xG_to_date
370,1920,38,2020-07-26,arsenal,2.3,3–2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,1658.365479,"['Emiliano Martínez', 'Kieran Tierney', 'Dani ...","['Ben Foster', 'Craig Dawson', 'Troy Deeney', ...",0.617,0.127,44.7,45.3
750,2021,38,2021-05-23,arsenal,0.8,2–0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,1728.190918,"['Bernd Leno', 'Kieran Tierney', 'Gabriel Dos ...","['Robert Sánchez', 'Ben White', 'Adam Webster'...",0.481,0.182,51.0,50.8
1130,2122,38,2022-05-22,arsenal,4.2,5–1,1.1,everton,Andre Marriner,4100d195,1846.874878,1709.672852,"['Aaron Ramsdale', 'Gabriel Dos Santos', 'Buka...","['Asmir Begović', 'Jonjoe Kenny', 'Mason Holga...",0.580,0.312,56.2,40.3
1510,2223,38,2023-05-28,arsenal,2.8,5–0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,1722.064575,"['Aaron Ramsdale', 'Ben White', 'Thomas Partey...","['José Sá', 'Nathan Collins', 'Mario Lemina', ...",0.987,0.216,68.7,36.6
1890,2324,38,2024-05-19,arsenal,2.9,2–1,0.6,everton,Michael Oliver,3e33bd98,1944.687744,1709.018433,"['David Raya', 'William Saliba', 'Ben White', ...","['Jordan Pickford', 'James Tarkowski', 'Dwight...",1.233,0.154,73.4,52.9


Now we are doing the same process, except for xG against.

In [10]:
# Assuming df is your original dataframe with the structure mentioned before
# Create two dataframes: one for the home team xG against and one for the away team xG against
home_xg_against_df = games[['season', 'date', 'home_team', 'away_xg']].rename(columns={'home_team': 'team', 'away_xg': 'xg_against'})
away_xg_against_df = games[['season', 'date', 'away_team', 'home_xg']].rename(columns={'away_team': 'team', 'home_xg': 'xg_against'})

# Concatenate the home and away xG against dataframes
all_xg_against_df = pd.concat([home_xg_against_df, away_xg_against_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_xg_against_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of xG against for each team, grouped by season
all_xg_against_df['xg_against_cumulative'] = all_xg_against_df.groupby(['team', 'season'])['xg_against'].cumsum()

# Shift the cumsum by one, so we don't include the xG against of the current game
all_xg_against_df['xg_against_cumulative'] = all_xg_against_df.groupby(['team', 'season'])['xg_against_cumulative'].shift(fill_value=0)

# Now we have the cumulative xG against for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_xg_against_df[['date', 'team', 'xg_against_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_against_cumulative': 'home_xG_against_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_xg_against_df[['date', 'team', 'xg_against_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_against_cumulative': 'away_xG_against_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_xG_against_to_date' and 'away_xG_against_to_date' columns with the correct cumulative xG against values


In [11]:
df.head()

,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,home_starters,away_starters,home_team_strength,away_team_strength,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date
0,1920,1,2019-08-09,liverpool,1.8,4–1,0.9,norwich city,Michael Oliver,928467bd,2050.401855,1634.853149,"['Alisson', 'Fabinho', 'Virgil van Dijk', 'Geo...","['Tim Krul', 'Max Aarons', 'Ben Godfrey', 'Gra...",1.103,0.000,0.0,0.0,0.0,0.0
1,1920,1,2019-08-10,bournemouth,1.3,1–1,1.3,sheffield utd,Kevin Friend,d402cacd,1695.075562,1623.745361,"['Aaron Ramsdale', 'Steve Cook', 'Nathan Aké',...","['Dean Henderson', 'George Baldock', 'Enda Ste...",0.212,0.000,0.0,0.0,0.0,0.0
2,1920,1,2019-08-10,burnley,0.9,3–0,1.2,southampton,Graham Scott,34b99058,1685.844604,1677.967529,"['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...","['Angus Gunn', 'Jannik Vestergaard', 'Jack Ste...",0.016,0.152,0.0,0.0,0.0,0.0
3,1920,1,2019-08-10,crystal palace,0.9,0–0,1.1,everton,Jonathan Moss,a802f51e,1746.714111,1773.843750,"['Vicente Guaita', 'Joel Ward', 'Patrick van A...","['Jordan Pickford', 'Michael Keane', 'Richarli...",0.173,0.281,0.0,0.0,0.0,0.0
4,1920,1,2019-08-10,tottenham,2.4,3–1,0.7,aston villa,Chris Kavanagh,404ee5d3,1894.996338,1616.385376,"['Hugo Lloris', 'Danny Rose', 'Toby Alderweire...","['Tom Heaton', 'Neil Taylor', 'John McGinn', '...",0.621,-0.017,0.0,0.0,0.0,0.0


### Goals Conceded and Scored to Date

Here we will complete a similar process to get goals scored and conceded to date for each team.

In [12]:
# Replace en dash '–' with standard hyphen '-' before splitting
df['score'] = df['score'].str.replace('–', '-', regex=False)

# Split the 'score' column into two new columns 'home_goals_scored' and 'away_goals_scored'
df[['home_goals_scored', 'away_goals_scored']] = df['score'].str.split('-', expand=True).astype(int)

# Now df has two new columns with the home and away goals as integers


In [13]:
df

,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,home_starters,away_starters,home_team_strength,away_team_strength,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored
0,1920,1,2019-08-09,liverpool,1.8,4-1,0.9,norwich city,Michael Oliver,928467bd,...,"['Alisson', 'Fabinho', 'Virgil van Dijk', 'Geo...","['Tim Krul', 'Max Aarons', 'Ben Godfrey', 'Gra...",1.103,0.000,0.0,0.0,0.0,0.0,4,1
1,1920,1,2019-08-10,bournemouth,1.3,1-1,1.3,sheffield utd,Kevin Friend,d402cacd,...,"['Aaron Ramsdale', 'Steve Cook', 'Nathan Aké',...","['Dean Henderson', 'George Baldock', 'Enda Ste...",0.212,0.000,0.0,0.0,0.0,0.0,1,1
2,1920,1,2019-08-10,burnley,0.9,3-0,1.2,southampton,Graham Scott,34b99058,...,"['Nick Pope', 'Matthew Lowton', 'Jack Cork', '...","['Angus Gunn', 'Jannik Vestergaard', 'Jack Ste...",0.016,0.152,0.0,0.0,0.0,0.0,3,0
3,1920,1,2019-08-10,crystal palace,0.9,0-0,1.1,everton,Jonathan Moss,a802f51e,...,"['Vicente Guaita', 'Joel Ward', 'Patrick van A...","['Jordan Pickford', 'Michael Keane', 'Richarli...",0.173,0.281,0.0,0.0,0.0,0.0,0,0
4,1920,1,2019-08-10,tottenham,2.4,3-1,0.7,aston villa,Chris Kavanagh,404ee5d3,...,"['Hugo Lloris', 'Danny Rose', 'Toby Alderweire...","['Tom Heaton', 'Neil Taylor', 'John McGinn', '...",0.621,-0.017,0.0,0.0,0.0,0.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2324,38,2024-05-19,crystal palace,2.5,5-0,0.9,aston villa,Darren Bond,c975c7a6,...,"['Dean Henderson', 'Tyrick Mitchell', 'Marc Gu...","['Robin Olsen', 'Diego Carlos', 'Ezri Konsa', ...",0.387,0.387,46.2,62.6,51.3,57.5,5,0
1896,2324,38,2024-05-19,liverpool,4.5,2-0,0.5,wolves,Chris Kavanagh,d4823ed5,...,"['Alisson', 'Wataru Endo', 'Virgil van Dijk', ...","['José Sá', 'Rayan Aït-Nouri', 'Santiago Bueno...",1.122,0.225,83.3,46.5,45.3,63.2,2,0
1897,2324,38,2024-05-19,luton town,2.0,2-4,1.1,fulham,Matt Donohue,0fde9d70,...,"['Thomas Kaminski', 'Gabriel Osho', 'Chiedozie...","['Bernd Leno', 'Kenny Tete', 'Calvin Bassey', ...",-0.017,0.207,40.3,49.7,76.7,61.0,2,4
1898,2324,38,2024-05-19,manchester city,1.9,3-1,0.4,west ham,John Brooks,29335211,...,"['Stefan Ortega', 'Kyle Walker', 'Rúben Dias',...","['Alphonse Areola', 'Aaron Cresswell', 'Kurt Z...",1.685,0.455,78.7,52.1,35.0,69.4,3,1


In [14]:
# Assuming df is your original dataframe and you already have 'home_goals_scored' and 'away_goals_scored' columns
# Create a unified dataframe for both home and away goals scored values
games_goals = df[['season', 'date', 'home_team', 'home_goals_scored', 'away_team', 'away_goals_scored']].copy()

# Create two dataframes: one for the home teams' goals and one for the away teams' goals
home_goals_df = games_goals[['season', 'date', 'home_team', 'home_goals_scored']].rename(columns={'home_team': 'team', 'home_goals_scored': 'goals'})
away_goals_df = games_goals[['season', 'date', 'away_team', 'away_goals_scored']].rename(columns={'away_team': 'team', 'away_goals_scored': 'goals'})

# Concatenate the home and away dataframes
all_goals_df = pd.concat([home_goals_df, away_goals_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_goals_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of goals for each team, grouped by season
all_goals_df['goals_cumulative'] = all_goals_df.groupby(['team', 'season'])['goals'].cumsum()

# Shift the cumsum by one, so we don't include the goals of the current game
all_goals_df['goals_cumulative'] = all_goals_df.groupby(['team', 'season'])['goals_cumulative'].shift(fill_value=0)

# Now we have the cumulative goals for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_goals_df[['date', 'team', 'goals_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_cumulative': 'home_goals_scored_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_goals_df[['date', 'team', 'goals_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_cumulative': 'away_goals_scored_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_goals_scored_to_date' and 'away_goals_scored_to_date' columns with the correct cumulative goals values


In [15]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,home_team_strength,away_team_strength,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date
370,1920,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,...,0.617,0.127,44.7,45.3,51.9,55.8,3,2,53,34
750,2021,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,...,0.481,0.182,51.0,50.8,42.8,34.6,2,0,53,40
1130,2122,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,...,0.580,0.312,56.2,40.3,44.5,51.0,5,1,56,42
1510,2223,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,...,0.987,0.216,68.7,36.6,41.7,57.6,5,0,83,31
1890,2324,38,2024-05-19,arsenal,2.9,2-1,0.6,everton,Michael Oliver,3e33bd98,...,1.233,0.154,73.4,52.9,27.2,52.4,2,1,89,39


In [16]:
# Assuming df is your original dataframe and you already have 'home_goals_scored' and 'away_goals_scored' columns
# Create a unified dataframe for both home and away goals conceded values
games_goals = df[['season', 'date', 'home_team', 'away_goals_scored', 'away_team', 'home_goals_scored']].copy()

# For goals conceded, we take the goals scored by the opposite team
# Create two dataframes: one for the home teams' goals conceded and one for the away teams' goals conceded
home_goals_conceded_df = games_goals[['season', 'date', 'home_team', 'away_goals_scored']].rename(columns={'home_team': 'team', 'away_goals_scored': 'goals_conceded'})
away_goals_conceded_df = games_goals[['season', 'date', 'away_team', 'home_goals_scored']].rename(columns={'away_team': 'team', 'home_goals_scored': 'goals_conceded'})

# Concatenate the home and away dataframes
all_goals_conceded_df = pd.concat([home_goals_conceded_df, away_goals_conceded_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_goals_conceded_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of goals conceded for each team, grouped by season
all_goals_conceded_df['goals_conceded_cumulative'] = all_goals_conceded_df.groupby(['team', 'season'])['goals_conceded'].cumsum()

# Shift the cumsum by one, so we don't include the goals conceded of the current game
all_goals_conceded_df['goals_conceded_cumulative'] = all_goals_conceded_df.groupby(['team', 'season'])['goals_conceded_cumulative'].shift(fill_value=0)

# Now we have the cumulative goals conceded for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_goals_conceded_df[['date', 'team', 'goals_conceded_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_conceded_cumulative': 'home_goals_conceded_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_goals_conceded_df[['date', 'team', 'goals_conceded_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_conceded_cumulative': 'away_goals_conceded_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_goals_conceded_to_date' and 'away_goals_conceded_to_date' columns with the correct cumulative goals conceded values


In [17]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date
370,1920,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,...,44.7,45.3,51.9,55.8,3,2,53,34,46,61
750,2021,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,...,51.0,50.8,42.8,34.6,2,0,53,40,39,44
1130,2122,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,...,56.2,40.3,44.5,51.0,5,1,56,42,47,61
1510,2223,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,...,68.7,36.6,41.7,57.6,5,0,83,31,43,53
1890,2324,38,2024-05-19,arsenal,2.9,2-1,0.6,everton,Michael Oliver,3e33bd98,...,73.4,52.9,27.2,52.4,2,1,89,39,28,49


In [18]:
df.columns

Index(['season', 'week', 'date', 'home_team', 'home_xg', 'score', 'away_xg',
       'away_team', 'referee', 'game_id', 'home_team_elo', 'away_team_elo',
       'home_starters', 'away_starters', 'home_team_strength',
       'away_team_strength', 'home_xG_to_date', 'away_xG_to_date',
       'home_xG_against_to_date', 'away_xG_against_to_date',
       'home_goals_scored', 'away_goals_scored', 'home_goals_scored_to_date',
       'away_goals_scored_to_date', 'home_goals_conceded_to_date',
       'away_goals_conceded_to_date'],
      dtype='object')

## Points to Date

We will again execute a similar process for results. 3 points for a win, 1 for a draw and 0 for a loss. The points will again be presented in a 'to date' fashion to avoid data leakage.

In [19]:

def calculate_points(home_goals, away_goals):
    if home_goals > away_goals:
        return (3, 0)  # Home team wins
    elif home_goals < away_goals:
        return (0, 3)  # Away team wins
    else:
        return (1, 1)  # Draw
# Apply the function to each row and create new columns for home and away match points
df[['home_match_points', 'away_match_points']] = df.apply(
    lambda row: calculate_points(row['home_goals_scored'], row['away_goals_scored']),
    axis=1, result_type='expand'
)

# Create a unified dataframe for both home and away points values
points_df = df[['season', 'date', 'home_team', 'home_match_points', 'away_team', 'away_match_points']].copy()

# Create two dataframes: one for the home teams and one for the away teams
home_points_df = points_df[['season', 'date', 'home_team', 'home_match_points']].rename(columns={'home_team': 'team', 'home_match_points': 'points'})
away_points_df = points_df[['season', 'date', 'away_team', 'away_match_points']].rename(columns={'away_team': 'team', 'away_match_points': 'points'})

# Concatenate the home and away dataframes
all_points_df = pd.concat([home_points_df, away_points_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_points_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of points for each team, grouped by season
all_points_df['points_cumulative'] = all_points_df.groupby(['team', 'season'])['points'].cumsum()

# Shift the cumsum by one, so we don't include the points of the current game
all_points_df['points_cumulative'] = all_points_df.groupby(['team', 'season'])['points_cumulative'].shift(fill_value=0)

# Merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_points_df[['season', 'date', 'team', 'points_cumulative']],
    left_on=['season', 'date', 'home_team'],
    right_on=['season', 'date', 'team'],
    how='left'
)
df.rename(columns={'points_cumulative': 'home_points_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_points_df[['season', 'date', 'team', 'points_cumulative']],
    left_on=['season', 'date', 'away_team'],
    right_on=['season', 'date', 'team'],
    how='left'
)
df.rename(columns={'points_cumulative': 'away_points_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Your dataframe now has 'home_points_to_date' and 'away_points_to_date' columns with the correct cumulative points values, reset for each season and not including the current game.


In [20]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date
370,1920,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,...,3,2,53,34,46,61,3,0,53,34
750,2021,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,...,2,0,53,40,39,44,3,0,58,41
1130,2122,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,...,5,1,56,42,47,61,3,0,66,39
1510,2223,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,...,5,0,83,31,43,53,3,0,81,41
1890,2324,38,2024-05-19,arsenal,2.9,2-1,0.6,everton,Michael Oliver,3e33bd98,...,2,1,89,39,28,49,3,0,86,48


In [21]:
df.columns

Index(['season', 'week', 'date', 'home_team', 'home_xg', 'score', 'away_xg',
       'away_team', 'referee', 'game_id', 'home_team_elo', 'away_team_elo',
       'home_starters', 'away_starters', 'home_team_strength',
       'away_team_strength', 'home_xG_to_date', 'away_xG_to_date',
       'home_xG_against_to_date', 'away_xG_against_to_date',
       'home_goals_scored', 'away_goals_scored', 'home_goals_scored_to_date',
       'away_goals_scored_to_date', 'home_goals_conceded_to_date',
       'away_goals_conceded_to_date', 'home_match_points', 'away_match_points',
       'home_points_to_date', 'away_points_to_date'],
      dtype='object')

## Adding Form

Finally, we will calculate form based on the points columns we just created. We selected the last 5 games played in the premier league as our period over which measure form. We took the total number of points accumulated over that period and then divided that number by 5 to give a value for form representing average points per match.

In [22]:

# Function to calculate form for the last 5 games for a given team and season
def calculate_form(team, season, df):
    # Filter matches for the current team and season
    team_matches = df[(df['season'] == season) & ((df['home_team'] == team) | (df['away_team'] == team))]
    
    # Determine match points based on whether the team was home or away
    team_matches['match_points'] = team_matches.apply(
        lambda x: x['home_match_points'] if x['home_team'] == team else x['away_match_points'], axis=1
    )
    
    # Calculate the rolling sum of the last 5 games' points, not including the current match
    rolling_points = team_matches['match_points'].rolling(window=5, min_periods=1).sum().shift()
    
    # Get the last value of the rolling sum which represents the form of the last 5 matches
    last_value = rolling_points.iloc[-1] if not rolling_points.empty else 0
    
    # Return the form
    return last_value / 5

# Assuming 'df' is your DataFrame with match data
# Initialize the columns with zeros
df['home_form'] = 0
df['away_form'] = 0

# Loop over the dataframe and calculate form for each match
for index, row in df.iterrows():
    # Calculate home and away form if week is greater than 5, otherwise set to 0
    if row['week'] > 5:
        df.at[index, 'home_form'] = calculate_form(row['home_team'], row['season'], df)
        df.at[index, 'away_form'] = calculate_form(row['away_team'], row['season'], df)
    else:
        df.at[index, 'home_form'] = 0
        df.at[index, 'away_form'] = 0





In [23]:
# You can now check your DataFrame for the 'home_form' and 'away_form' columns
df[['week', 'home_team', 'away_team', 'season', 'home_form', 'away_form']].head(20)


,week,home_team,away_team,season,home_form,away_form
0,1,liverpool,norwich city,1920,0.0,0.0
1,1,bournemouth,sheffield utd,1920,0.0,0.0
2,1,burnley,southampton,1920,0.0,0.0
3,1,crystal palace,everton,1920,0.0,0.0
4,1,tottenham,aston villa,1920,0.0,0.0
5,1,watford,brighton,1920,0.0,0.0
6,1,west ham,manchester city,1920,0.0,0.0
7,1,leicester city,wolves,1920,0.0,0.0
8,1,manchester utd,chelsea,1920,0.0,0.0
9,1,newcastle utd,arsenal,1920,0.0,0.0


In [24]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 5]

arsenal_week38


,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date,home_form,away_form
48,1920,5,2019-09-15,watford,2.7,2-2,0.8,arsenal,Anthony Taylor,8257eda8,...,2,6,8,6,1,1,1,7,0.0,0.0
420,2021,5,2020-10-17,manchester city,1.3,1-0,0.9,arsenal,Chris Kavanagh,e95b8546,...,6,8,7,5,3,0,4,9,0.0,0.0
802,2122,5,2021-09-18,burnley,1.0,0-1,1.1,arsenal,Anthony Taylor,a427debc,...,3,1,8,9,0,3,1,3,0.0,0.0
1184,2223,5,2022-08-31,arsenal,2.4,2-1,0.4,aston villa,Robert Jones,cc235aad,...,11,3,3,7,3,0,12,3,0.0,0.0
1567,2324,5,2023-09-17,everton,0.3,0-1,1.0,arsenal,Simon Hooper,b1278924,...,2,8,8,4,0,3,1,10,0.0,0.0


In [25]:
# Conditions for the different classes based on 'home_match_points'
conditions = [
    df['home_match_points'] == 3,
    df['home_match_points'] == 1,
    df['home_match_points'] == 0,
]

# Corresponding class labels for the target column
class_labels = [0, 1, 2]

# Create the target column 'match_result'
df['match_result'] = np.select(conditions, class_labels)

# Now your DataFrame will have a 'match_result' column with values 0, 1, or 2 based on 'home_match_points'
print(df[['home_match_points', 'match_result']])


      home_match_points  match_result
0                     3             0
1                     1             1
2                     3             0
3                     1             1
4                     3             0
...                 ...           ...
1895                  3             0
1896                  3             0
1897                  0             2
1898                  3             0
1899                  0             2

[1900 rows x 2 columns]


In [26]:
df

,season,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,...,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date,home_form,away_form,match_result
0,1920,1,2019-08-09,liverpool,1.8,4-1,0.9,norwich city,Michael Oliver,928467bd,...,0,0,0,3,0,0,0,0.0,0.0,0
1,1920,1,2019-08-10,bournemouth,1.3,1-1,1.3,sheffield utd,Kevin Friend,d402cacd,...,0,0,0,1,1,0,0,0.0,0.0,1
2,1920,1,2019-08-10,burnley,0.9,3-0,1.2,southampton,Graham Scott,34b99058,...,0,0,0,3,0,0,0,0.0,0.0,0
3,1920,1,2019-08-10,crystal palace,0.9,0-0,1.1,everton,Jonathan Moss,a802f51e,...,0,0,0,1,1,0,0,0.0,0.0,1
4,1920,1,2019-08-10,tottenham,2.4,3-1,0.7,aston villa,Chris Kavanagh,404ee5d3,...,0,0,0,3,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2324,38,2024-05-19,crystal palace,2.5,5-0,0.9,aston villa,Darren Bond,c975c7a6,...,76,58,56,3,0,46,68,2.6,1.6,0
1896,2324,38,2024-05-19,liverpool,4.5,2-0,0.5,wolves,Chris Kavanagh,d4823ed5,...,50,41,63,3,0,79,46,1.6,0.6,0
1897,2324,38,2024-05-19,luton town,2.0,2-4,1.1,fulham,Matt Donohue,0fde9d70,...,51,81,59,0,3,26,44,0.2,1.0,2
1898,2324,38,2024-05-19,manchester city,1.9,3-1,0.4,west ham,John Brooks,29335211,...,59,33,71,3,0,88,52,3.0,0.8,0


In [27]:
csv_file_path = '/Users/lkimball/Desktop/Flatiron/CapstoneProject/EPL_Data_featured.csv'
df.to_csv(csv_file_path, index=False)